In [1]:
import requests
from kafka import KafkaProducer
import json

def send_plane_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from the api    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        
        # Publish each entry to Kafka
       
        for state in states:
            output = {}
            for key in fields.keys():
                output[key] = state[fields[key]]
            # output["icao24"] = state[0]  # Id of the plane  (ICAO24)
            # output["origin_country"] = state[2]
            # output["velocity"] = state[9]
            # output["timestamp"] = state[3]  # Timestamp de la dernière position
            # output["latitude"] = state[6]  # Latitude de l'avion
            # output["longitude"] = state[5]  # Longitude de l'avion
            producer.send(topic, value=output)
        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [3]:
from datetime import datetime, timedelta
fields = {
    "icao24":0,
    "origin_country":1,
    "velocity":9,
    "timestamp":3,
    "latitude":6,
    "longitude":5
}

api_url = "https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

In [5]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_plane_to_kafka("planes", api_url, fields)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-25 17:00:00-2025-01-25 18:00:00
Sent 558 records.
2025-01-25 18:00:00-2025-01-25 19:00:00
Sent 560 records.
2025-01-25 19:00:00-2025-01-25 20:00:00
Sent 559 records.
2025-01-25 20:00:00-2025-01-25 21:00:00
Sent 559 records.
2025-01-25 21:00:00-2025-01-25 22:00:00
Sent 559 records.
2025-01-25 22:00:00-2025-01-25 23:00:00
Sent 558 records.
2025-01-25 23:00:00-2025-01-26 00:00:00
Sent 558 records.
2025-01-26 00:00:00-2025-01-26 01:00:00
Sent 559 records.
2025-01-26 01:00:00-2025-01-26 02:00:00
Sent 558 records.
2025-01-26 02:00:00-2025-01-26 03:00:00
Sent 554 records.


In [7]:
import requests
response = requests.get("https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            origin_country = state[2]
            velocity = state[9]
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}, Velocity : {velocity}, Origin : {origin_country}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 511142, Timestamp: 1742295327, Latitude: 45.8174, Longitude: 8.7794, Velocity : 59.87, Origin : Estonia
ID: 4b1812, Timestamp: 1742295074, Latitude: 47.4518, Longitude: 8.5575, Velocity : 0, Origin : Switzerland
ID: 4b1805, Timestamp: 1742295327, Latitude: 47.7443, Longitude: 8.421, Velocity : 120.57, Origin : Switzerland
ID: 4b1800, Timestamp: 1742295323, Latitude: 47.448, Longitude: 8.559, Velocity : 0, Origin : Switzerland
ID: 4b1802, Timestamp: 1742295327, Latitude: 46.4211, Longitude: 7.2169, Velocity : 205, Origin : Switzerland
ID: 39de61, Timestamp: 1742295300, Latitude: 46.4113, Longitude: 5.7853, Velocity : 36.22, Origin : France
ID: 440dc2, Timestamp: 1742295328, Latitude: 48.8906, Longitude: 8.872, Velocity : 92.72, Origin : Austria
ID: 39de44, Timestamp: 1742295327, Latitude: 46.196, Longitude: 5.0235, Velocity : 232.39, Origin : France
ID: 4b180b, Timestamp: 1742295327, Latitude: 45.6393, Longitude: 5.5638, Velocity : 147.79, Origin : Switzerland
ID: 3c6669, Timestamp: